# 최종모델 저장

In [43]:
import pandas as pd
df = pd.read_csv("../Data/서울시 한강공원 시간별 주차대수.csv")

In [44]:
df.drop(columns='Unnamed: 0', inplace=True)

In [45]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, KFold

# 1. 데이터 준비
features = ['요일', '휴일여부', '주차장명', '연도', '월', '일']
target_usage = ['아침 이용시간', '낮 이용시간', '저녁 이용시간']
targets_parking = ['아침 주차대수', '낮 주차대수', '저녁 주차대수']

# 원본 데이터 복사
df_xgb = df.copy()

# 범주형 데이터 원-핫 인코딩
X_xgb = pd.get_dummies(df_xgb[features])
y_xgb_usage = df_xgb[target_usage]
y_xgb_parking = df_xgb[targets_parking]

# 원-핫 인코딩 결과를 float 타입으로 변환
X_xgb = X_xgb.astype(float)

# 2. 이용시간 예측 모델 학습 (KFold)
kf = KFold(n_splits=5, shuffle=True)
usage_train_scores = []
usage_test_scores = []

for train_index, test_index in kf.split(X_xgb):
    X_train_usage, X_test_usage = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    y_train_usage, y_test_usage = y_xgb_usage.iloc[train_index], y_xgb_usage.iloc[test_index]

    # 이용시간 모델 학습
    usage_model = MultiOutputRegressor(
        XGBRegressor(learning_rate=0.1, max_depth=6, n_estimators=300)
    )
    usage_model.fit(X_train_usage, y_train_usage)

    # 점수 저장
    usage_train_scores.append(usage_model.score(X_train_usage, y_train_usage))
    usage_test_scores.append(usage_model.score(X_test_usage, y_test_usage))

# 평균 점수 출력
mean_usage_train_score = sum(usage_train_scores) / len(usage_train_scores)
mean_usage_test_score = sum(usage_test_scores) / len(usage_test_scores)
print(f"이용시간 모델 평균 훈련 점수 (R²): {mean_usage_train_score:.4f}")
print(f"이용시간 모델 평균 테스트 점수 (R²): {mean_usage_test_score:.4f}")

# 최종 이용시간 모델 학습 및 저장
usage_model.fit(X_xgb, y_xgb_usage)
joblib.dump(usage_model, 'Yeouido_usage_model.pkl')

# 예측된 이용시간 추가
df_xgb['예측 아침 이용시간'] = usage_model.predict(X_xgb)[:, 0]
df_xgb['예측 낮 이용시간'] = usage_model.predict(X_xgb)[:, 1]
df_xgb['예측 저녁 이용시간'] = usage_model.predict(X_xgb)[:, 2]

# 3. 주차대수 예측 모델 학습 (train_test_split)
features_with_usage = ['예측 아침 이용시간', '예측 낮 이용시간', '예측 저녁 이용시간'] + features
X_parking = pd.get_dummies(df_xgb[features_with_usage])

X_train_parking, X_test_parking, y_train_parking, y_test_parking = train_test_split(
    X_parking, y_xgb_parking, test_size=0.2
)

parking_model = MultiOutputRegressor(
    XGBRegressor(learning_rate=0.1, max_depth=6, n_estimators=300)
)
parking_model.fit(X_train_parking, y_train_parking)

# 모델 및 피처 순서 저장
joblib.dump(parking_model, 'Yeouido_parking_model.pkl')
joblib.dump(X_xgb.columns.tolist(), 'Yeouido_features_usage.pkl')  # 이용시간 피처 저장
joblib.dump(X_parking.columns.tolist(), 'Yeouido_features_parking.pkl')  # 주차대수 피처 저장

print(f"주차대수 모델 훈련 점수: {parking_model.score(X_train_parking, y_train_parking):.4f}")
print(f"주차대수 모델 테스트 점수: {parking_model.score(X_test_parking, y_test_parking):.4f}")

이용시간 모델 평균 훈련 점수 (R²): 0.9565
이용시간 모델 평균 테스트 점수 (R²): 0.8772
주차대수 모델 훈련 점수: 0.9887
주차대수 모델 테스트 점수: 0.9212
